# Check the feasibility

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## Hyperparameters

In [2]:
EPOCHS = 10

DATASET_PATH = 'dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## Setting the simplest model

In [3]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(64, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

## Importing the dataset (preprocess)

In [4]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img)
    return tf.image.convert_image_dtype(img, tf.float32)

In [5]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train & Valid dataset

In [6]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

## Generating the model and training

In [7]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Train for 21 steps, validate for 9 steps
Epoch 1/10
21/21 [==============================] - 81s 4s/step - loss: 0.4461 - accuracy: 0.8290 - val_loss: 0.3943 - val_accuracy: 0.8627
Epoch 2/10
21/21 [==============================] - 67s 3s/step - loss: 0.3674 - accuracy: 0.8699 - val_loss: 0.2578 - val_accuracy: 0.9155
Epoch 3/10
21/21 [==============================] - 88s 4s/step - loss: 0.3588 - accuracy: 0.8775 - val_loss: 0.3741 - val_accuracy: 0.8697
Epoch 4/10
21/21 [==============================] - 98s 5s/step - loss: 0.3303 - accuracy: 0.8865 - val_loss: 0.3814 - val_accuracy: 0.8662
Epoch 5/10
21/21 [==============================] - 110s 5s/step - loss: 0.3664 - accuracy: 0.8684 - val_loss: 0.3532 - val_accuracy: 0.8838
Epoch 6/10
21/21 [==============================] - 101s 5s/step - loss: 0.3706 - accuracy: 0.8699 - val_loss: 0.3695 - val_accuracy: 0.8556
Epoch 7/10
21/21 [==============================] - 117s 6s/step - loss: 0.3482 - accuracy: 0.8744 - val_loss: 0.3285

## Saving the result (as the image)

In [9]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index +=1